In [1]:
import requests
import pandas as pd
import numpy as np
import keys
import json
from tqdm import tqdm

In [2]:
states = list(pd.read_csv('states.csv').Abbreviation)

In [3]:
frame_list = []
success = []
fail = []
for state in tqdm(states):
    for page in range(1,30):
        try:
            url ='https://api.seatgeek.com/2/events?venue.state='
            req = requests.get(f'{url}{state}&page={page}&type=concert&client_id={keys.seatgeek}')
            x = json.loads(req.text)
            df = pd.DataFrame(x['events'])
            frame_list.append(df)
            success.append(f'{state},{page}')
        except:
            fail.append(f'{state},{page}')

100%|██████████| 51/51 [04:09<00:00,  4.48s/it]


In [4]:
seatgeek = pd.concat(frame_list)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
seatgeek.reset_index(inplace=True)

In [6]:
seatgeek.columns

Index(['index', 'access_method', 'announce_date', 'announcements',
       'conditional', 'created_at', 'date_tbd', 'datetime_local',
       'datetime_tbd', 'datetime_utc', 'description', 'enddatetime_utc',
       'event_promotion', 'general_admission', 'id', 'is_open', 'links',
       'performers', 'popularity', 'score', 'short_title', 'stats', 'status',
       'taxonomies', 'time_tbd', 'title', 'type', 'url', 'venue',
       'visible_until_utc'],
      dtype='object')

In [7]:
seatgeek.performers[0]

[{'image': 'https://seatgeek.com/images/performers-landscape/vandoliers-ddab46/409987/huge.jpg',
  'primary': True,
  'colors': None,
  'images': {'huge': 'https://seatgeek.com/images/performers-landscape/vandoliers-ddab46/409987/huge.jpg'},
  'has_upcoming_events': True,
  'id': 409987,
  'stats': {'event_count': 5},
  'image_license': 'https://creativecommons.org/licenses/by/2.0/',
  'score': 0.5,
  'location': None,
  'taxonomies': [{'name': 'concerts',
    'parent_id': None,
    'id': 2000000,
    'document_source': {'source_type': 'ELASTIC', 'generation_type': 'FULL'}}],
  'type': 'band',
  'num_upcoming_events': 5,
  'short_name': 'Vandoliers',
  'home_venue_id': None,
  'slug': 'vandoliers',
  'divisions': None,
  'name': 'Vandoliers',
  'url': 'https://seatgeek.com/vandoliers-tickets',
  'popularity': 0,
  'image_attribution': 'https://www.flickr.com/photos/arekolek/7847640482/'}]

In [8]:
seatgeek.drop(columns = ['index', 'announce_date','access_method','description','taxonomies','announcements', 'conditional', 'date_tbd','datetime_tbd', 'enddatetime_utc',
                         'event_promotion','type','time_tbd','links','status','is_open','datetime_utc', 'visible_until_utc','title'],inplace=True)

In [9]:
seatgeek['listing_count']=seatgeek.stats.apply(lambda x: x['listing_count'])
seatgeek['average_price']=seatgeek.stats.apply(lambda x: x['average_price'])
seatgeek['median_price']=seatgeek.stats.apply(lambda x: x['median_price'])
seatgeek['lowest_price']=seatgeek.stats.apply(lambda x: x['lowest_price'])
seatgeek.drop(columns=['stats'], inplace =True)

In [10]:
seatgeek.venue[0]

{'links': [],
 'metro_code': 0,
 'postal_code': '35205',
 'timezone': 'America/Chicago',
 'has_upcoming_events': True,
 'id': 479986,
 'city': 'Birmingham',
 'extended_address': 'Birmingham, AL 35205',
 'display_location': 'Birmingham, AL',
 'state': 'AL',
 'score': 0,
 'location': {'lat': 33.5054, 'lon': -86.7905},
 'access_method': None,
 'num_upcoming_events': 3,
 'address': '2514 10th Ave S',
 'capacity': 0,
 'slug': 'the-nick',
 'name': 'The Nick',
 'url': 'https://seatgeek.com/venues/the-nick/tickets',
 'country': 'US',
 'popularity': 0,
 'name_v2': 'The Nick'}

In [11]:
seatgeek['venue_name'] = seatgeek.venue.apply(lambda x: x['name'])
seatgeek['venue_popularity'] = seatgeek.venue.apply(lambda x: x['popularity'])
seatgeek['venue_address'] = seatgeek.venue.apply(lambda x: x['address'])
seatgeek['venue_city'] = seatgeek.venue.apply(lambda x: x['city'])
seatgeek['venue_state'] = seatgeek.venue.apply(lambda x: x['state'])
seatgeek['venue_zip'] = seatgeek.venue.apply(lambda x: x['postal_code'])
seatgeek['venue_id'] = seatgeek.venue.apply(lambda x: x['id'])
seatgeek.drop(columns=['venue'],inplace=True)

In [12]:
seatgeek.general_admission.fillna(False, inplace=True)

In [15]:
seatgeek.columns

Index(['created_at', 'datetime_local', 'general_admission', 'id', 'performers',
       'popularity', 'score', 'short_title', 'url', 'listing_count',
       'average_price', 'median_price', 'lowest_price', 'venue_name',
       'venue_popularity', 'venue_address', 'venue_city', 'venue_state',
       'venue_zip', 'venue_id'],
      dtype='object')

In [16]:
seatgeek.drop(columns = ['created_at', 'datetime_local', 'id','url'],inplace=True)

In [23]:
seatgeek['headliner']= seatgeek.performers.apply(lambda x: x[0]['short_name'])

In [27]:
seatgeek.drop(columns=['performers'],inplace=True)

In [29]:
seatgeek.drop(columns=['score'],inplace=True)

In [30]:
seatgeek.columns

Index(['general_admission', 'popularity', 'short_title', 'listing_count',
       'average_price', 'median_price', 'lowest_price', 'venue_name',
       'venue_popularity', 'venue_address', 'venue_city', 'venue_state',
       'venue_zip', 'venue_id', 'headliner'],
      dtype='object')

In [32]:
seatgeek.rename(columns={'general_admission':'is_ga','popularity':'popularity_sg'},inplace=True)

In [38]:
seatgeek['is_ga']=seatgeek.is_ga.astype('float')

In [41]:
seatgeek.to_csv('sg_events.csv')